I have tried creating new files for every output that will help us in tracking the effeiciency of the codes. Till now, most of the codes work quite fine. However, it takes a long time for the codes to work on the files

The computer might also give warnings of High Memory Alert. One can simply ignore them and continue with their execution of the codes. The codes are written with reference from the Perl Script given in the Google Drive

# Description: These 2 following codes shows how to transform the raw data to a useable format. I have added 2 different codes. In case one doesn't works, one can try using the 2nd one

In [ ]:
import re
import os

def clean_and_prepare_10k(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

        # Remove header and irrelevant sections
        cleaned_text = re.sub(r'UNITED STATES SECURITIES AND EXCHANGE COMMISSION[\s\S]+?(?=EXPLANATORY NOTE)', '', text)

        # Extract relevant sections
        sections = re.findall(r'(ITEM \d+\..*?)(?=ITEM \d+\.|\Z)', cleaned_text, re.DOTALL)

        # Format sections
        formatted_sections = []
        for section in sections:
            # Remove extra whitespace and newlines
            section = section.strip()
            # Remove section indicators like 'ITEM X.'
            section = re.sub(r'ITEM \d+\.', '', section)
            formatted_sections.append(section)

    return formatted_sections

def save_formatted_sections(file_path, sections):
    file_dir = os.path.dirname(file_path)
    file_name = os.path.basename(file_path)
    new_file_name = os.path.splitext(file_name)[0] + '_formatted.txt'
    new_file_path = os.path.join(file_dir, new_file_name)
    with open(new_file_path, 'w', encoding='utf-8') as new_file:
        for i, section in enumerate(sections, 1):
            new_file.write(f"Section {i}:\n{section}\n{'-'*50}\n")
    print(f"Formatted sections saved to: {new_file_path}")

# Example usage:
file_path = 'naya/naya2/naya3/target.txt'
sections = clean_and_prepare_10k(file_path)
save_formatted_sections(file_path, sections)

# Print the formatted sections
for i, section in enumerate(sections, 1):
    print(f"Section {i}:\n{section}\n{'-'*50}\n")

In [ ]:
import os
import re
import shutil
from html.parser import HTMLParser

def trim(string):
    return string.strip()

def trimend(string):
    return string.rstrip() + '\n'

class MyHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.data = []
        self.is_html = False

    def handle_starttag(self, tag, attrs):
        if tag.lower() == 'html':
            self.is_html = True

    def handle_data(self, data):
        self.data.append(data)

    def get_text(self):
        return ''.join(self.data)

def alter_html_code_to_character(line):
    # Alter encoded HTML characters to convert to proper characterset.
    html_codes = {
        '&#32;': ' ', '&#33;': '!', '&#34;': '"', '&#35;': '#', '&#36;': '$', '&#37;': '%', '&#38;': '&', '&#39;': "'",
        '&#40;': '(', '&#41;': ')', '&#42;': '*', '&#43;': '+', '&#44;': ',', '&#45;': '-', '&#46;': '.', '&#47;': '/',
        '&#48;': '0', '&#49;': '1', '&#50;': '2', '&#51;': '3', '&#52;': '4', '&#53;': '5', '&#54;': '6', '&#55;': '7',
        '&#56;': '8', '&#57;': '9', '&#58;': ':', '&#59;': ';', '&#60;': '<', '&#61;': '=', '&#62;': '>', '&#63;': '?',
        '&#64;': '@', '&#65;': 'A', '&#66;': 'B', '&#67;': 'C', '&#68;': 'D', '&#69;': 'E', '&#70;': 'F', '&#71;': 'G',
        '&#72;': 'H', '&#73;': 'I', '&#74;': 'J', '&#75;': 'K', '&#76;': 'L', '&#77;': 'M', '&#78;': 'N', '&#79;': 'P',
        '&#80;': 'P', '&#81;': 'Q', '&#82;': 'R', '&#83;': 'S', '&#84;': 'T', '&#85;': 'U', '&#86;': 'V', '&#87;': 'W',
        '&#88;': 'X', '&#89;': 'Y', '&#90;': 'Z', '&#91;': '[', '&#92;': '\\', '&#93;': ']', '&#94;': '^', '&#95;': '_',
        '&#96;': '`', '&#97;': 'a', '&#98;': 'b', '&#99;': 'c', '&#100;': 'd', '&#101;': 'e', '&#102;': 'f', '&#103;': 'g',
        '&#104;': 'h', '&#105;': 'i', '&#106;': 'j', '&#107;': 'k', '&#108;': 'l', '&#109;': 'm', '&#110;': 'n', '&#111;': 'o',
        '&#112;': 'p', '&#113;': 'q', '&#114;': 'r', '&#115;': 's', '&#116;': 't', '&#117;': 'u', '&#118;': 'v', '&#119;': 'w',
        '&#120;': 'x', '&#121;': 'y', '&#122;': 'z', '&#123;': '{', '&#124;': '|', '&#125;': '}', '&#126;': '~', '&#145;': "'",
        '&#146;': "'", '&#147;': '"', '&#148;': '"', '&#160;': ' ', '&#161;': '¡', '&#162;': '¢', '&#163;': '£', '&#164;': '¤',
        '&#165;': '¥', '&#166;': '¦', '&#167;': '§', '&#168;': '¨', '&#169;': '©', '&#170;': 'ª', '&#171;': '«', '&#172;': '¬',
        '&#174;': '®', '&#175;': '¯', '&#176;': '°', '&#177;': '±', '&#178;': '²', '&#179;': '³', '&#180;': '´', '&#181;': 'µ',
        '&#182;': '¶', '&#183;': '·', '&#184;': '¸', '&#185;': '¹', '&#186;': 'º', '&#187;': '»', '&#188;': '¼', '&#189;': '½',
        '&#190;': '¾', '&#191': '¿', '&#8211;': '–', '&#8212;': '—', '&#8216;': '‘', '&#8217;': '’', '&#8218;': '‚', '&#8220;': '"',
        '&#8221;': '"', '&#8364;': '€', '&#8482;': '™', '&ndash;': '–', '&mdash;': '—', '&iexcl;': '¡', '&iquest;': '¿',
        '&quot;': '"', '&laquo;': '«', '&raquo;': '»', '&nbsp;': ' ', '&amp;': '&', '&cent;': '¢', '&copy;': '©', '&divide;': '÷',
        '&gt;': '>', '&lt;': '<', '&middot;': '·', '&para;': '¶', '&plusmn;': '±', '&euro;': '€', '&pound;': '£', '&reg;': '®',
        '&sect;': '§', '&yen;': '¥', '&aacute;': 'á', '&Aacute;': 'Á', '&agrave;': 'à', '&Agrave;': 'À', '&acirc;': 'â',
        '&Acirc;': 'Â', '&aring;': 'å', '&Aring;': 'Å', '&atilde;': 'ã', '&Atilde;': 'Ã', '&auml;': 'ä', '&Auml;': 'Ä',
        '&aelig;': 'æ', '&AElig;': 'Æ', '&ccedil;': 'ç', '&Ccedil;': 'Ç', '&eacute;': 'é', '&Eacute;': 'É', '&egrave;': 'è',
        '&Egrave;': 'È', '&ecirc;': 'ê', '&Ecirc;': 'Ê', '&euml;': 'ë', '&Euml;': 'Ë', '&iacute;': 'í', '&Iacute;': 'Í',
        '&igrave;': 'ì', '&Igrave;': 'Ì', '&icirc;': 'î', '&Icirc;': 'Î', '&iuml;': 'ï', '&Iuml;': 'Ï', '&ntilde;': 'ñ',
        '&Ntilde;': 'Ñ', '&oacute;': 'ó', '&Oacute;': 'Ó', '&ograve;': 'ò', '&Ograve;': 'Ò', '&ocirc;': 'ô', '&Ocirc;': 'Ô',
        '&oslash;': 'ø', '&Oslash;': 'Ø', '&otilde;': 'õ', '&Otilde;': 'Õ', '&ouml;': 'ö', '&Ouml;': 'Ö', '&szlig;': 'ß',
        '&uacute;': 'ú', '&Uacute;': 'Ú', '&ugrave;': 'ù', '&Ugrave;': 'Ù', '&ucirc;': 'û', '&Ucirc;': 'Û', '&uuml;': 'ü',
        '&Uuml;': 'Ü', '&yuml;': 'ÿ', '&quot;': '"', '&apos;': "'", '&amp;': '&', '&lt;': '<', '&gt;': '>', '&nbsp;': ' ',
        '&iexcl;': '¡', '&cent;': '¢', '&pound;': '£', '&curren;': '¤', '&yen;': '¥', '&brvbar;': '¦', '&sect;': '§',
        '&uml;': '¨', '&copy;': '©', '&ordf;': 'ª', '&laquo;': '«', '&not;': '¬', '&shy;': '­', '&reg;': '®', '&macr;': '¯',
        '&deg;': '°', '&plusmn;': '±', '&sup2;': '²', '&sup3;': '³', '&acute;': '´', '&micro;': 'µ', '&para;': '¶',
        '&middot;': '·', '&cedil;': '¸', '&sup1;': '¹', '&ordm;': 'º', '&raquo;': '»', '&frac14;': '¼', '&frac12;': '½',
        '&frac34;': '¾', '&iquest;': '¿', '&times;': '×', '&divide;': '÷', '&Agrave;': 'À', '&Aacute;': 'Á', '&Acirc;': 'Â',
        '&Atilde;': 'Ã', '&Auml;': 'Ä', '&Aring;': 'Å', '&AElig;': 'Æ', '&Ccedil;': 'Ç', '&Egrave;': 'È', '&Eacute;': 'É',
        '&Ecirc;': 'Ê', '&Euml;': 'Ë', '&Igrave;': 'Ì', '&Iacute;': 'Í', '&Icirc;': 'Î', '&Iuml;': 'Ï', '&ETH;': 'Ð',
        '&Ntilde;': 'Ñ', '&Ograve;': 'Ò', '&Oacute;': 'Ó', '&Ocirc;': 'Ô', '&Otilde;': 'Õ', '&Ouml;': 'Ö', '&times;': '×',
        '&Oslash;': 'Ø', '&Ugrave;': 'Ù', '&Uacute;': 'Ú', '&Ucirc;': 'Û', '&Uuml;': 'Ü', '&Yacute;': 'Ý', '&THORN;': 'Þ',
        '&szlig;': 'ß', '&agrave;': 'à', '&aacute;': 'á', '&acirc;': 'â', '&atilde;': 'ã', '&auml;': 'ä', '&aring;': 'å',
        '&aelig;': 'æ', '&ccedil;': 'ç', '&egrave;': 'è', '&eacute;': 'é', '&ecirc;': 'ê', '&euml;': 'ë', '&igrave;': 'ì',
        '&iacute;': 'í', '&icirc;': 'î', '&iuml;': 'ï', '&eth;': 'ð', '&ntilde;': 'ñ', '&ograve;': 'ò', '&oacute;': 'ó',
        '&ocirc;': 'ô', '&otilde;': 'õ', '&ouml;': 'ö', '&divide;': '÷', '&oslash;': 'ø', '&ugrave;': 'ù', '&uacute;': 'ú',
        '&ucirc;': 'û', '&uuml;': 'ü', '&yacute;': 'ý', '&thorn;': 'þ', '&yuml;': 'ÿ', '&quot;': '"', '&amp;': '&',
        '&apos;': "'", '&lt;': '<', '&gt;': '>', '&nbsp;': ' ', '&iexcl;': '¡', '&cent;': '¢', '&pound;': '£',
        '&curren;': '¤', '&yen;': '¥', '&brvbar;': '¦', '&sect;': '§', '&uml;': '¨', '&copy;': '©', '&ordf;': 'ª',
        '&laquo;': '«', '&not;': '¬', '&shy;': '­', '&reg;': '®', '&macr;': '¯', '&deg;': '°', '&plusmn;': '±',
        '&sup2;': '²', '&sup3;': '³', '&acute;': '´', '&micro;': 'µ', '&para;': '¶', '&middot;': '·', '&cedil;': '¸',
        '&sup1;': '¹', '&ordm;': 'º', '&raquo;': '»', '&frac14;': '¼', '&frac12;': '½', '&frac34;': '¾', '&iquest;': '¿',
        '&times;': '×', '&divide;': '÷'
    }

    for code, char in html_codes.items():
        line = line.replace(code, char)
        print("done")
    return line

def clean_html_file(source_html, target_text):
    parser = MyHTMLParser()
    with open(source_html, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            parser.feed(line)
            text = parser.get_text()
            text = trim(text)
            text = trimend(text)
            
            if text:
                print(text,"\n")
                try:
                    with open(target_text, 'w', encoding='utf-8') as out_file:
                        out_file.write(text)
                        out_file.write('\n')
                except Exception as e:
                    print(f"An error occurred while appending to the file: {e}")  
                    
source_file = "C:\\Users\\Pravalika Das\\DATA_WALA\\UPDATED_NLP_COURSE\\SEC EDGAR\\naya\\naya2\\naya3\\SEC.txt"
target_file = "C:\\Users\\Pravalika Das\\DATA_WALA\\UPDATED_NLP_COURSE\\SEC EDGAR\\naya\\naya2\\naya3\\target2.txt"

clean_html_file(source_file, target_file)

# You might want to add more code here to verify if the cleaning process was successful or to handle exceptions.
print("HTML file cleaned and saved to text.")


# Description: This script finds and extracts the footnotes section from the cleaned 10-K filings text files;

This code displays the raw data that was converted to the formatted data to a proper footnotes kind of file with extra spaces, etc.

In [ ]:
import os
import re
import shutil

# Define regular expressions for header information
header_formtype = re.compile(r'^\s*FORM\s*TYPE:\s*(.*)')
header_cik = re.compile(r'CENTRAL INDEX KEY:\s{1,}([0-9]{10})\s{0,}')
header_filedate = re.compile(r'FILED AS OF DATE:\s{1,}([0-9]{8})\s{0,}')
header_fiscdate = re.compile(r'CONFORMED PERIOD OF REPORT:\s{1,}([0-9]{8})\s{0,}')
header_sic = re.compile(r'STANDARD INDUSTRIAL CLASSIFICATION:\s{1,}.*([0-9]{4})\]{0,}\s{0,}')

# Define input and output directories
INPUT_DIR = "naya/naya2/naya3/SEC2"
OUTPUT_DIR = "naya/naya2/naya3/SEC2"

# Function to trim whitespace from both ends of a string
def trim(s):
    return s.strip()

# Function to process a text file
def process_file(file_path):
    print("Processing file:", file_path)
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Process each line
    cleaned_lines = []
    for line in lines:
        cleaned_line = line.strip()  # Remove leading and trailing whitespace
        # Additional processing if needed
        cleaned_lines.append(cleaned_line)

    # Write cleaned lines to a new file
    output_file_path = os.path.join(OUTPUT_DIR, os.path.basename(file_path))
    with open(output_file_path, 'w',encoding='utf-8') as output_file:
        output_file.write('\n'.join(cleaned_lines))

    print("File processed and saved to:", output_file_path)

# Main function
def main():
    # Create the output directory if it doesn't exist
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    # Iterate over files in the input directory
    for filename in os.listdir(INPUT_DIR):
        if filename.endswith(".txt"):
            file_path = os.path.join(INPUT_DIR, filename)
            process_file(file_path)

if __name__ == "__main__":
    main()

# Description: This script gets the sentences from the footnotes

This code removes all the extra spaces and arranges the contents of the files in the form of sentences in the file

In [ ]:
import os
import re
from nltk.tokenize import sent_tokenize

# Change these working directories and files as needed
OUTPUTDIR = "naya/naya2/naya3/SEC2"

# Regular Expressions
page = r'(^\s*?\n)*(^\s*?(page\s*?)?-?\d{1,3}-?\s*?\n)(^\s*?\n)*'
pageof = r'(^\s*?\n)*(^\s*?(page\s*?)?\s*?\d+\s*?of\s*?\d+?\s*?\n)(^\s*?\n)*'
toc = r'(^\s*?\n)*(^\s*?\[\d+\]\s*Table\sof\sContents\s*\n)(^\s*?\n)*'
fpage = r'(^\s*?\n)*(^\s*?F-\d+\s*\n)(^\s*?\n)*'
continued = r'(^\s*?\n)*(^\s*?.{0,60}?\((continued|cont.|cont)\)\s*\n)(^\s*?\n)*'

# Process all files in the directory
for file in os.listdir():
    if file.endswith("sentences.txt"):
        print(f"Processing file: {file}")
        sentences_file = file.replace("10KNotes", "10KNotesSentences")
        output_file_path = os.path.join(OUTPUTDIR, sentences_file)

        if os.path.exists(output_file_path):
            print(f"File already exists: {output_file_path}")
            continue
        
        with open(file, 'r') as input_file, open(output_file_path, 'w') as output_file:
            data = input_file.read()
            # Apply regular expressions
            data = re.sub(page, '', data)
            data = re.sub(pageof, '', data)
            data = re.sub(toc, '', data)
            data = re.sub(fpage, '', data)
            data = re.sub(continued, '', data)

            # Sentence segmentation
            sentences = sent_tokenize(data)
            for sentence in sentences:
                output_file.write(sentence.strip() + '\n')

        print(f"Processed file saved at: {output_file_path}")

# Description: This file contains the regular expressions to find the footnote sections, the lists are in order of priority


Here, the code can be modified to find the words and the sentence where the word occurs. One can simply copy paste any 1 of the categories and change the word(example: AI) to the desired word(example: appear, communications,etc)

In [ ]:
import re

# Define categories and their corresponding regular expressions
categories = {
    "AI": [
        re.compile(r'AI(?:s)?', re.IGNORECASE)
    ],
    "subs": [
        re.compile(r'subsidiaries(?:s)?', re.IGNORECASE)
    ],
    "MICRO": [
        re.compile(r'Microsoft(?:s)?', re.IGNORECASE)
    ],
    # Add more categories and their regular expressions here...
}

def match_categories(line):
    matches = []
    for category, regex_list in categories.items():
        for regex in regex_list:
            if regex.search(line):
                matches.append(category)
                break
    return matches

def process_file(input_file, output_file):
    with open(input_file, 'r') as f_input, open(output_file, 'w') as f_output:
        for line in f_input:
            matched_categories = match_categories(line)
            if matched_categories:
                f_output.write(f"{line.strip()} - Matches: {', '.join(matched_categories)}\n")

def main():
    input_file = 'naya/naya2/naya3/SEC2/target_sentences.txt'
    output_file = 'naya/naya2/naya3/SEC2/target1.txt'
    process_file(input_file, output_file)

if __name__ == "__main__":
    main()


# Description: This script counts the number of estimation words in the notes to the financial statements

In case, one wants to know the number of occurences of a word, this code will provide the count.

In the code below,
COR[P|PORATION|PORATIONS]: This part shows that words with CORP or CORPORATION or CORPORATIONS are to be counted. Similarly other words can be written and searched.

Example: appear[s|ing|ed]: this tells the code to search for words containing appear or appearing or appeared and to tell the count of each.

In [ ]:
import os
import re

def trim(string):
    return string.strip()

# Working directory
input_directory = "C:\\Users\\Pravalika Das\\DATA_WALA\\UPDATED_NLP_COURSE\\SEC EDGAR\\naya\\naya2\\naya3\\SEC2\\SEC3\\SEC4\\SEC5"
# Output file
output_file = open("C:\\Users\\Pravalika Das\\DATA_WALA\\UPDATED_NLP_COURSE\\SEC EDGAR\\naya\\naya2\\naya3\\SEC2\\SEC3\\SEC4\\untitled_format1.txt", "w", encoding='utf-8')

# Dictionary to store word counts
word_counts = {}

# Loop through files in directory
for file_name in os.listdir(input_directory):
    if file_name.endswith(".txt") or file_name.endswith(".TXT"):
        print("Processing file...", file_name)
        try:
            with open(file_name, 'r', encoding='utf-8') as temp:
                for line in temp:
                    words = line.split()
                    for word in words:
                        # Check if the word is similar to "appear" using regex
                        if re.match(r'COR[P|PORATION|PORATIONS]*', word, re.IGNORECASE):
                            word = word.lower()  # Convert to lowercase for consistency
                            word_counts[word] = word_counts.get(word, 0) + 1
        except FileNotFoundError:
            print("File doesn't exist... moving on")
            continue

# Output word counts to the file
for word, count in word_counts.items():
    output_file.write(f"{word}: {count}\n")

output_file.close()


# Description: Parser code similar to the one given in JAVA Part

The following code shows the Parsed file of the actual file. This code was written with reference to the JAVA Code that was provided in the Google Drive.

In [ ]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Function to parse file and store results in another file
def parse_file(file_path, output_file):
    with open(file_path, 'r', encoding='latin-1') as file, open(output_file, 'a', encoding='utf-8') as output:
        # Skip header line
        next(file)
        
        # Process each line
        for line in file:
            line = line.strip()
            if len(line) <= 425:  # Length check to eliminate outlier formatting errors
                output.write(line + '\n')
                doc = nlp(line)
                # Write dependencies to output file
                for token in doc:
                    output.write(f"{token.text} {token.dep_} {token.head.text} {token.head.pos_} "
                                 f"{[child for child in token.children]}\n")

# Input and output directories
input_directory = "C:\\Users\\Pravalika Das\\DATA_WALA\\UPDATED_NLP_COURSE\\SEC EDGAR\\naya\\naya2\\naya3\\SEC2\\SEC3"
output_directory = "C:\\Users\\Pravalika Das\\DATA_WALA\\UPDATED_NLP_COURSE\\SEC EDGAR\\naya\\naya2\\naya3\\SEC2\\SEC3"

# Create output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Iterate over files in input directory
import os
for file_name in os.listdir(input_directory):
    if file_name.endswith(".txt") or file_name.endswith(".TXT"):
        input_file_path = os.path.join(input_directory, file_name)
        output_file_path = os.path.join(output_directory, f"parsed_{file_name}")
        print("Processing Input File... ", file_name)
        parse_file(input_file_path, output_file_path)
